<a href="https://colab.research.google.com/github/LongNguyen1984/DeepLearning/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [17]:
from pandas import read_csv
from pandas import DataFrame
import numpy as np

def load_data(filename):
  dataset = read_csv(filename, header=0);
  data = dataset.values;
  return data
def segment_data(signal, distance, overlap):
  i = 1;
  s = list()
  while int(i+distance)<signal.shape[0]:
    a = signal[int(i):int(i+distance)];
    #a = [x for x in t]
    #a = DataFrame.transpose(DataFrame(t))
    #a = np.split(a,int(distance-1)
    s.append(a)
    i = int(i + distance*(1-overlap))
    
  return np.asarray(s) # return a nd array

lenW = 100
ol = 0.5
data = load_data('drive/My Drive/InvertPhaseLong2.csv')  
ppg = segment_data(data[:,0],lenW, ol)
ppg1 = segment_data(data[:,2],lenW, ol)
#ppg2 = segment_data(data[:,1],lenW, ol)

data2 = load_data('drive/My Drive/Dataset/longcunDB2.csv')
ppg2 = segment_data(data2[:,1],lenW, ol)

In [16]:
# label for first group of Data
y1 = [1]*ppg.shape[0];
y2 = [0]*ppg1.shape[0];
y3 = [2]*ppg2.shape[0];

y1 = np.asarray(y1)
y2 = np.asarray(y2)
y3 = np.asarray(y3)

#ydf1 = DataFrame(y1)
#ydf2 = DataFrame(y2)
# Concatenate data
X = np.concatenate((ppg, ppg1,ppg2))
y = np.concatenate((y1, y2, y3))
print(X.shape)

# rescale data
from sklearn.preprocessing import MinMaxScaler
  
# define min max scaler
def scaling(signal):
  scaler = MinMaxScaler()
  s_trans = signal.transpose()
  Xscaled = scaler.fit_transform(s_trans)
  Xscaled = Xscaled.transpose()
  return Xscaled  
# Scaling Input Data 
Xscaled = scaling(X)
Xscaled2 = scaling(ppg2)

(1236, 100)


Repearing Data

In [28]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(Xscaled, y, test_size=0.33, random_state=5)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

#X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
#X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))
# X_val = X_val.reshape((X_val.shape[0],X_val.shape[1],1))
y_train = to_categorical(y_train, num_classes= 3)
y_test = to_categorical(y_test, num_classes= 3)
# y_val = to_categorical(y_val, num_classes= 3)

# Build Model MLP

In [39]:
# lstm model for the har dataset
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical


def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 50, 64
  #n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  n_timesteps, n_outputs = X_train.shape[1], y_train.shape[1]
  model = Sequential()
  model.add(Dense(100, activation='relu',input_dim=n_timesteps))
  #model.add(Dense(100, activation='relu', input_shape=(n_timesteps,n_features)))
  model.add(Dense(200, activation='relu'))
  model.add(Dense(100, activation='relu'))
  #model.add(Dropout(0.5))
  #model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy, history

# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=10):
  # load data
  #trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    
    score, history = evaluate_model(X_train, y_train, X_test, y_test)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)
  return history

# run the experiment
history = run_experiment()

>#1: 99.265
>#2: 99.265
>#3: 99.265
>#4: 99.510
>#5: 100.000
>#6: 99.510
>#7: 99.510
>#8: 99.510
>#9: 99.755
>#10: 99.510
[99.26470518112183, 99.26470518112183, 99.26470518112183, 99.50980544090271, 100.0, 99.50980544090271, 99.50980544090271, 99.50980544090271, 99.75489974021912, 99.50980544090271]
Accuracy: 99.510% (+/-0.219)


In [49]:
verbose, epochs, batch_size = 0, 25, 32
#n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
n_timesteps, n_outputs = X_train.shape[1], y_train.shape[1]
model = Sequential()
#model.add(Bidirectional(LSTM(100,activation='relu'), input_shape=(n_timesteps,n_features)))
model.add(Dense(100, activation='relu',input_dim=n_timesteps))
  #model.add(Dense(50, activation='relu'))
  #model.add(Dropout(0.5))
  #model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

In [23]:
X_train.shape

(828, 100, 1)

In [48]:
model.summary()

Model: "sequential_125"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_356 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_357 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_358 (Dense)            (None, 3)                 153       
Total params: 15,303
Trainable params: 15,303
Non-trainable params: 0
_________________________________________________________________


# K-Fold Cross Validation

In [50]:
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical

num_folds = 10
SEED = 34
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)
skf = StratifiedKFold(n_splits=num_folds, random_state=SEED,shuffle=True)
sss = StratifiedShuffleSplit(n_splits=num_folds, random_state=SEED, test_size=3)
#Reshape X, y
#Xrsh = Xscaled.reshape((Xscaled.shape[0],Xscaled.shape[1],1))
#yrsh = to_categorical(y, num_classes=3)

Xrsh = X_train
yrsh = y_train

# fit network
#history = model.fit(Xrsh, yrsh, validation_split=0.33, epochs=epochs, batch_size=batch_size, verbose=verbose)

# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []
fold_no = 1


for train, test in kfold.split(Xrsh, yrsh):
#for train, test in skf.split(Xrsh, yrsh):
#for train, test in sss.split(Xrsh, yrsh):
  # Build the model
  verbose, epochs, batch_size = 1, 25, 32
  n_timesteps, n_outputs = Xrsh.shape[1], yrsh.shape[1]
  model = Sequential()
  model.add(Dense(100, activation='relu',input_dim=n_timesteps))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  # Fit data to model
  history = model.fit(Xrsh[train], yrsh[train], epochs=epochs, batch_size=batch_size, verbose=verbose)
  
  # Generate generalization metrics
  scores = model.evaluate(Xrsh[test], yrsh[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/25
24/24 [==============================] - 0s 2ms/step - loss: 1.0141 - accuracy: 0.5208
Epoch 2/25
24/24 [==============================] - 0s 2ms/step - loss: 0.8291 - accuracy: 0.7275
Epoch 3/25
24/24 [==============================] - 0s 2ms/step - loss: 0.6928 - accuracy: 0.7678
Epoch 4/25
24/24 [==============================] - 0s 2ms/step - loss: 0.5636 - accuracy: 0.8362
Epoch 5/25
24/24 [==============================] - 0s 2ms/step - loss: 0.4400 - accuracy: 0.8685
Epoch 6/25
24/24 [==============================] - 0s 2ms/step - loss: 0.3605 - accuracy: 0.8980
Epoch 7/25
24/24 [==============================] - 0s 2ms/step - loss: 0.2825 - accuracy: 0.9221
Epoch 8/25
24/24 [==============================] - 0s 2ms/step - loss: 0.2248 - accuracy: 0.9530
Epoch 9/25
24/24 [==============================] - 0s 2ms/step - loss: 0.1812 - accuracy: 0.9705
Epoch 10/25
24/24 [==

In [44]:
accuracy= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 0s 2ms/step - loss: 0.0274 - accuracy: 0.9902
